In [1]:
import torch
import mlflow
import gc

from visualize_image import visualize_one_xy_slice_in_3d_image
from loading_dataset import load_data
from model import load_model
import numpy as np
from monai.data import box_utils
from monai.apps.detection.metrics.matching import matching_batch
from warmup_scheduler import GradualWarmupScheduler
from monai.apps.detection.metrics.coco import COCOMetric

In [2]:
import os

os.environ['MLFLOW_TRACKING_USERNAME'] = 'Eddy'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'Usp1#'
os.environ['MLFLOW_EXPERIMENT_NAME'] = 'HC'
description = "HC test to verify if there is anything wrong with HC and MSD"

In [3]:
gt_box_mode = 'cccwhd'
batch_size = 8
patch_size = [96,96,40]
data_list_file_path = '/data/output/hc_train_val3.json'
# data_list_file_path = '/data/output/msd_train_val3.json'
# data_list_file_path = '/data/output/LUNA16_datasplit/dataset_fold0.json'
# data_list_file_path = '/data/output/LUNA16-mini.json'
data_base_dir = '/data/HC_Images_resample/'
# data_base_dir = '/data/MSD_Images_resample/'
# data_base_dir = '/data/LUNA16_Images_resample/'
amp=True

returned_layers = [1,2]
base_anchor_shapes = [[6,8,4],[8,6,5],[10,10,6]]
conv1_t_stride = [2,2,1]
n_input_channels = 1
spatial_dims = 3
fg_labels = [0]
verbose = False
balanced_sampler_pos_fraction = 0.3
score_thresh = 0.02
nms_thresh = 0.22
val_patch_size = [256,256,104]

lr = 1e-2
val_interval = 1
# val_interval = 5
coco_metric = COCOMetric(classes=["nodule"], iou_list=[0.1], max_detection=[100])
best_val_epoch_metric = 0.0
best_val_epoch = -1
max_epochs = 20
w_cls = 1.0

compute_dtype = torch.float32
if amp:
    compute_dtype = torch.float16

# monai.config.print_config()
torch.backends.cudnn.benchmark = True

### loading dataset

In [4]:
train_loader, val_loader, len_train_ds = load_data(
    gt_box_mode, patch_size, batch_size, amp, data_list_file_path, data_base_dir
)

monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.


### loading model

In [5]:
detector, device = load_model(
    returned_layers, base_anchor_shapes, conv1_t_stride, n_input_channels,
    spatial_dims, fg_labels, verbose, balanced_sampler_pos_fraction,
    score_thresh, nms_thresh, val_patch_size
)

### Initialize training

In [6]:
optimizer = torch.optim.SGD(
    detector.network.parameters(),
    lr,
    momentum=0.9,
    weight_decay=3e-5,
    nesterov=True
)

after_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=150, gamma=0.1
)
scheduler_warmup = GradualWarmupScheduler(
    optimizer, multiplier=1, total_epoch=10, after_scheduler=after_scheduler
)
scaler = torch.cuda.amp.GradScaler() if amp else None
optimizer.zero_grad()
optimizer.step()

### train

In [7]:
epoch_len = len_train_ds // train_loader.batch_size

with mlflow.start_run(description=description) as run:

    mlflow.log_param("gt_box_mode", gt_box_mode)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("patch_size", patch_size)
    mlflow.log_param("data_list_file_path", data_list_file_path)
    mlflow.log_param("data_base_dir", data_base_dir)
    mlflow.log_param("amp", amp)
    
    mlflow.log_param("n_input_channels", n_input_channels)
    mlflow.log_param("spatial_dims", spatial_dims)
    mlflow.log_param("balanced_sampler_pos_fraction", balanced_sampler_pos_fraction)
    mlflow.log_param("score_thresh", score_thresh)
    mlflow.log_param("nms_thresh", nms_thresh)
    
    mlflow.log_param("initial_lr", lr)
    mlflow.log_param("val_interval", val_interval)
    mlflow.log_param("max_epochs", max_epochs)
    mlflow.log_param("w_cls", w_cls)
    
    for epoch in range(max_epochs):
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")
        detector.train()
        epoch_loss = 0
        epoch_cls_loss = 0
        epoch_box_reg_loss = 0
        step = 0
        scheduler_warmup.step()

        for batch_data in train_loader:
            step += 1
            inputs = [
                batch_data_ii["image"].to(device) for batch_data_i in batch_data for batch_data_ii in batch_data_i
            ]
            targets = [
                dict(
                    label=batch_data_ii["label"].to(device),
                    box=batch_data_ii["box"].to(device)
                )
                for batch_data_i in batch_data
                for batch_data_ii in batch_data_i
            ]

            for param in detector.network.parameters():
                param.grad = None

            if amp and (scaler is not None):
                with torch.cuda.amp.autocast():
                    outputs = detector(inputs, targets)
                    loss = w_cls * outputs[detector.cls_key] + outputs[detector.box_reg_key]
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = detector(inputs, targets)
                loss = w_cls * outputs[detector.cls_key] + outputs[detector.box_reg_key]
                loss.backward()
                optimizer.step()

            # saving into mlflow
            epoch_loss += loss.detach().item()
            epoch_cls_loss += outputs[detector.cls_key].detach().item()
            epoch_box_reg_loss += outputs[detector.box_reg_key].detach().item()
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            mlflow.log_metric("train_loss", loss.detach().item(), epoch_len * epoch + step)

        del inputs, batch_data
        torch.cuda.empty_cache()
        gc.collect()

        epoch_loss /= step
        epoch_cls_loss /= step
        epoch_box_reg_loss /= step
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
        mlflow.log_metric("avg_train_loss", epoch_loss, epoch + 1)
        mlflow.log_metric("avg_train_cls_loss", epoch_cls_loss, epoch + 1)
        mlflow.log_metric("avg_train_box_reg_loss", epoch_box_reg_loss, epoch + 1)
        mlflow.log_metric("train_lr", optimizer.param_groups[0]["lr"], epoch + 1)

        # saving last trained model
        mlflow.pytorch.log_model(detector.network, "model")

        # validation for model selection
        if (epoch + 1) % val_interval == 0:
            detector.eval()
            val_outputs_all = []
            val_targets_all = []

            with torch.no_grad():
                for val_data in val_loader:
                    # if all val_data_i["image"] smaller than val_patch_size, no need to use inferer
                    # otherwise, need inferer to handle large input images.
                    use_inferer = not all(
                        [val_data_i["image"][0, ...].numel() < np.prod(val_patch_size) for val_data_i in val_data]
                    )
                    val_inputs = [val_data_i.pop("image").to(device) for val_data_i in val_data]

                    if amp:
                        with torch.cuda.amp.autocast():
                            val_outputs = detector(val_inputs, use_inferer=use_inferer)
                    else:
                        val_outputs = detector(val_inputs, use_inferer=use_inferer)

                    # save outputs for evaluation
                    val_outputs_all += val_outputs
                    val_targets_all += val_data

            # visualize an inference image and boxes
            print(val_data[0]["image_meta_dict"]["filename_or_obj"])
            draw_img = visualize_one_xy_slice_in_3d_image(
                gt_boxes=val_data[0]["box"].cpu().detach().numpy(),
                image=val_inputs[0][0, ...].cpu().detach().numpy(),
                pred_boxes=val_outputs[0][detector.target_box_key].cpu().detach().numpy(),
            )
            # mlflow.log_image(draw_img.transpose([2, 1, 0]), "val_img_xy.png")
            mlflow.log_image(draw_img, str(epoch + 1) + "_val_img_xy.png")

            # compute metrics
            del val_inputs
            torch.cuda.empty_cache()
            results_metric = matching_batch(
                iou_fn=box_utils.box_iou,
                iou_thresholds=coco_metric.iou_thresholds,
                pred_boxes=[
                    val_data_i[detector.target_box_key].cpu().detach().numpy() for val_data_i in val_outputs_all
                ],
                pred_classes=[
                    val_data_i[detector.target_label_key].cpu().detach().numpy() for val_data_i in val_outputs_all
                ],
                pred_scores=[
                    val_data_i[detector.pred_score_key].cpu().detach().numpy() for val_data_i in val_outputs_all
                ],
                gt_boxes=[val_data_i[detector.target_box_key].cpu().detach().numpy() for val_data_i in val_targets_all],
                gt_classes=[
                    val_data_i[detector.target_label_key].cpu().detach().numpy() for val_data_i in val_targets_all
                ]
            )
            val_epoch_metric_dict = coco_metric(results_metric)[0]
            print(val_epoch_metric_dict)

            # write metrics
            for k in val_epoch_metric_dict.keys():
                mlflow.log_metric("val_" + k, val_epoch_metric_dict[k], epoch + 1)
            val_epoch_metric = val_epoch_metric_dict.values()
            val_epoch_metric = sum(val_epoch_metric) / len(val_epoch_metric)
            mlflow.log_metric("val_metric", val_epoch_metric, epoch + 1)

            # save best trained model
            if val_epoch_metric > best_val_epoch_metric:
                best_val_epoch_metric = val_epoch_metric
                best_val_epoch = epoch + 1
                mlflow.pytorch.log_model(detector.network, "best_model")
            print(
                "current epoch: {} current metric: {:.4f} "
                "best metric: {:.4f} at epoch {}".format(
                    epoch + 1, val_epoch_metric, best_val_epoch_metric, best_val_epoch
                )
            )

    print(f"train completed, best_metric: {best_val_epoch_metric:.4f} " f"at epoch: {best_val_epoch}")

----------
epoch 1/20
1/60, train_loss: 0.7134
2/60, train_loss: 1.2832
3/60, train_loss: 0.9443
4/60, train_loss: 1.5117
5/60, train_loss: 1.0391
6/60, train_loss: 1.1523
7/60, train_loss: 0.4619
8/60, train_loss: 1.0127
9/60, train_loss: 0.9893
10/60, train_loss: 0.8125
11/60, train_loss: 1.1787
12/60, train_loss: 0.9697
13/60, train_loss: 0.8242
14/60, train_loss: 1.2637
15/60, train_loss: 0.9790
16/60, train_loss: 1.2891
17/60, train_loss: 0.9551
18/60, train_loss: 0.6221
19/60, train_loss: 0.9746
20/60, train_loss: 0.6826
21/60, train_loss: 0.4941
22/60, train_loss: 0.6235
23/60, train_loss: 0.8032
24/60, train_loss: 0.5576
25/60, train_loss: 0.5537
26/60, train_loss: 0.6440
27/60, train_loss: 1.0801
28/60, train_loss: 0.6094
29/60, train_loss: 0.4963
30/60, train_loss: 0.8018
31/60, train_loss: 0.5977
32/60, train_loss: 0.6553
33/60, train_loss: 0.6694
34/60, train_loss: 0.7676
35/60, train_loss: 0.4778
36/60, train_loss: 0.8003
37/60, train_loss: 0.3823
38/60, train_loss: 0.9795

2023/09/17 13:48:45 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/17 13:48:50 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


/data/HC_Images_resample/PL259838368407140/PL259838368407140.nii.gz
{'mAP_IoU_0.10_0.50_0.05_MaxDet_100': 0.0, 'nodule_mAP_IoU_0.10_0.50_0.05_MaxDet_100': 0.0, 'AP_IoU_0.10_MaxDet_100': 0.0, 'nodule_AP_IoU_0.10_MaxDet_100': 0.0, 'mAR_IoU_0.10_0.50_0.05_MaxDet_100': 0.0, 'nodule_mAR_IoU_0.10_0.50_0.05_MaxDet_100': 0.0, 'AR_IoU_0.10_MaxDet_100': 0.0, 'nodule_AR_IoU_0.10_MaxDet_100': 0.0}
current epoch: 1 current metric: 0.0000 best metric: 0.0000 at epoch -1
----------
epoch 2/20
1/60, train_loss: 0.9907
2/60, train_loss: 0.5869
3/60, train_loss: 0.8262
4/60, train_loss: 0.6055
5/60, train_loss: 0.8042
6/60, train_loss: 1.1787
7/60, train_loss: 0.3784
8/60, train_loss: 0.6836
9/60, train_loss: 0.6895
10/60, train_loss: 0.4426
11/60, train_loss: 0.4631
12/60, train_loss: 0.4678
13/60, train_loss: 0.9580
14/60, train_loss: 1.3994
15/60, train_loss: 0.5181
16/60, train_loss: 0.8633
17/60, train_loss: 0.6421
18/60, train_loss: 0.4448
19/60, train_loss: 0.5654
20/60, train_loss: 0.8560
21/60,

2023/09/17 14:20:20 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/17 14:20:23 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


/data/HC_Images_resample/PL259838368407140/PL259838368407140.nii.gz
{'mAP_IoU_0.10_0.50_0.05_MaxDet_100': 0.38755875920960875, 'nodule_mAP_IoU_0.10_0.50_0.05_MaxDet_100': 0.38755875920960875, 'AP_IoU_0.10_MaxDet_100': 0.4239423977266444, 'nodule_AP_IoU_0.10_MaxDet_100': 0.4239423977266444, 'mAR_IoU_0.10_0.50_0.05_MaxDet_100': 0.7222222222222222, 'nodule_mAR_IoU_0.10_0.50_0.05_MaxDet_100': 0.7222222222222222, 'AR_IoU_0.10_MaxDet_100': 0.75, 'nodule_AR_IoU_0.10_MaxDet_100': 0.75}


...

current epoch: 19 current metric: 0.8683 best metric: 0.8683 at epoch 19
----------
epoch 20/20
1/60, train_loss: 0.1400
2/60, train_loss: 0.1213
3/60, train_loss: 0.1700
4/60, train_loss: 0.1405
5/60, train_loss: 0.1332
6/60, train_loss: 0.1718
7/60, train_loss: 0.2056
8/60, train_loss: 0.0997
9/60, train_loss: 0.1213
10/60, train_loss: 0.2544
11/60, train_loss: 0.3137
12/60, train_loss: 0.1284
13/60, train_loss: 0.2588
14/60, train_loss: 0.1440
15/60, train_loss: 0.1307
16/60, train_loss: 0.4817
17/60, train_loss: 0.1227
18/60, train_loss: 0.1028
19/60, train_loss: 0.1469
20/60, train_loss: 0.1418
21/60, train_loss: 0.6143
22/60, train_loss: 0.3682
23/60, train_loss: 0.1638
24/60, train_loss: 0.2009
25/60, train_loss: 0.1670
26/60, train_loss: 0.1936
27/60, train_loss: 0.2383
28/60, train_loss: 0.3755
29/60, train_loss: 0.1560
30/60, train_loss: 0.1455
31/60, train_loss: 0.1606
32/60, train_loss: 0.3770
33/60, train_loss: 0.1484
34/60, train_loss: 0.1791
35/60, train_loss: 0.1602
36/

2023/09/18 00:04:45 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/18 00:04:48 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/18 00:04:51 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under /mlflow/3/8b9205029a4f43f2812fc95b5999a625/artifacts. Set logging level to DEBUG via 

/data/HC_Images_resample/PL259838368407140/PL259838368407140.nii.gz
{'mAP_IoU_0.10_0.50_0.05_MaxDet_100': 0.8074807529926825, 'nodule_mAP_IoU_0.10_0.50_0.05_MaxDet_100': 0.8074807529926825, 'AP_IoU_0.10_MaxDet_100': 0.9504950524556754, 'nodule_AP_IoU_0.10_MaxDet_100': 0.9504950524556754, 'mAR_IoU_0.10_0.50_0.05_MaxDet_100': 0.8888888888888888, 'nodule_mAR_IoU_0.10_0.50_0.05_MaxDet_100': 0.8888888888888888, 'AR_IoU_0.10_MaxDet_100': 1.0, 'nodule_AR_IoU_0.10_MaxDet_100': 1.0}


2023/09/18 00:08:57 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/18 00:09:00 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/18 00:09:04 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under /mlflow/3/8b9205029a4f43f2812fc95b5999a625/artifacts. Set logging level to DEBUG via 

current epoch: 20 current metric: 0.9117 best metric: 0.9117 at epoch 20
train completed, best_metric: 0.9117 at epoch: 20
